In [144]:
from itertools import product
import numpy as np
import random

In [145]:
user_states = [-1, 0, 1]
items = [-1, 1]
slates = list(product(items, repeat=5))
num_actions=len(slates)

In [146]:
def reward_slateq(u,r):
    rew=u*r
    return rew
def reward(u,r):
    rew=0
    for j in range(len(r)):
        if u!=0:
            rew+=u*r[j]
        else:
            rew+=0.0
    if rew>=0:
        rew=1
    elif rew==0:
        rew=0
    else:
        rew=-1
    return rew

In [147]:
def transition_function_slateq(current_state, action):
    # Update the new state based on the action
    if action == None:
        new_state = 0
    else:
        new_state = action
    return new_state
def transition_function(current_state, action):
    # Update the new state based on the action
    new_state = current_state + sum(action)
    
    # Clip the new state to ensure it stays within the defined states (1, -1, 0)
    new_state = max(min(new_state, 1), -1)
    
    return new_state

In [148]:
prob_i_u = np.zeros((len(user_states), len(items)))
for i in range(len(user_states)):
    for j in range(len(items)):
        if i == j:
            prob_i_u[i, j] = 0.6
        else:
            prob_i_u[i, j] = random.uniform(0, 0.5)

In [149]:
Q_item = np.zeros((len(user_states), len(items)))
Q_slate = np.zeros((len(user_states), len(slates)))
Q_table = np.zeros((len(user_states), len(slates)))

In [150]:
def randSlate(s=0,otherargs=[]):
    i = np.random.choice(num_actions)
    action = slates[i]
   
    return action

In [151]:
def qLearned(s=0,qtable=[]):
    st = user_states.index(s)
    action_indx = np.argmax(qtable[st,:])
    slate = slates[action_indx]
    return slate

In [152]:
# def update_q_value_slateq(Q_item,Q_slate, current_state, selected_item, slate, reward_value, next_state,alpha, gamma):
#     state=user_states.index(current_state)
#     action=slates.index(slate)
#     if selected_item!=0:
#         selected_item_index=items.index(selected_item)
#         current_q_value=Q_item[state,selected_item_index]
#         next_state=user_states.index(next_state)
#         max_next_q_value=np.max(Q_item[next_state,:])
#         new_q_value=current_q_value+alpha*(reward_value+gamma*max_next_q_value-current_q_value)
#         Q_item[state,selected_item_index]=new_q_value
    
#     for i in range(len(slate)):
#         item_action=items.index(slate[i])
#         Q_slate[state,action]+=Q_item[state,item_action]*prob_i_u[state,item_action]

#     return Q_item, Q_slate



In [153]:
def epsilon_greedy_action_slateq(Q_slate, current_state, epsilon):
    state=user_states.index(current_state)
    if np.random.rand() < epsilon:
        # Choose a random action

        i = np.random.choice(num_actions)
        action = slates[i]
    else:
        # Choose the action with the highest Q-value
        i = np.argmax(Q_slate[state])
        action = slates[i]
    return action
def epsilon_greedy_action(Q_table, current_state, epsilon):
    state=user_states.index(current_state)
    if np.random.rand() < epsilon:
        # Choose a random action

        i = np.random.choice(num_actions)
        action = slates[i]
    else:
        # Choose the action with the highest Q-value
        i = np.argmax(Q_table[state])
        action = slates[i]
    return action

In [154]:
def softmax(x):
    exp_x = np.exp(x)
    softmax_values = exp_x / np.sum(exp_x)
    return softmax_values

In [155]:
def run_episodes_slateq(Q_slateq,serveSlate=randSlate,num_episodes=1000,  session_length=10, transition_function_slateq=transition_function_slateq, reward_slateq=reward_slateq, epsilon=0.01):
    training=[]
    cumr=0
    for episode in range(num_episodes):
        # Randomly choose an initial state
        current_state = np.random.choice(user_states)
        sess_reward=0
        
        for sess in range(session_length):
            # Choose an action using epsilon-greedy strategy
            selected_slate = serveSlate(current_state,Q_slateq)
            prob_selection=[]
            for i in range(len(selected_slate)):
                prob_selection.append(prob_i_u[user_states.index(current_state),items.index(selected_slate[i])])
            prob_selection.append(0.2)
            prob_selection=softmax(prob_selection)
            selected_index = np.random.choice(len(prob_selection), p=prob_selection)
            if selected_index<= len(selected_slate)-1:
                selected_item=selected_slate[selected_index]
            else:
                selected_item=0 #None
            
            
            next_state = transition_function_slateq(current_state, selected_item)
            
            # Assume a reward for the transition (you should replace this with the actual reward from your environment)
            reward_value = reward_slateq(current_state,selected_item)
             # Replace with the actual reward
            training.append([current_state,selected_item,selected_slate, reward_value,next_state])
            # # Update the Q-value based on the transition
            # Q_item,Q_slate = update_q_value_slateq(Q_item,Q_slate, current_state,selected_item,selected_slate, reward_value, next_state,alpha, gamma)
            sess_reward+=reward_value
            # Move to the next state
            current_state = next_state
            
            # # Terminate the episode if a terminal state is reached (you should replace this with your termination condition)
            # if current_state in [-1, 1]:
   #     break
        cumr+=sess_reward
    return cumr, training


def run_episodes(Q_table,serveSlate=randSlate, num_episodes=1000, session_length=10, transition_function=transition_function, reward=reward,epsilon=0.1):
    training=[]
    cumr=0
    for episode in range(num_episodes):
        # Randomly choose an initial state
        current_state = np.random.choice(user_states)
        sess_reward=0
        
        for sess in range(session_length):
            
            # Choose an action using epsilon-greedy strategy
            selected_action = serveSlate(current_state,Q_table)
            action=slates.index(selected_action)
            # Transition to the next state based on the chosen action
            next_state = transition_function(current_state, selected_action)
            
            # Assume a reward for the transition (you should replace this with the actual reward from your environment)
            reward_value = reward(current_state,selected_action)
            sess_reward+=reward_value # Replace with the actual reward
            training.append([current_state,selected_action,next_state,reward_value])
            # Update the Q-value based on the transition
            # Q_table = update_q_value(Q_table, current_state, selected_action, reward_value, next_state,alpha, gamma)
            
            # Move to the next state
            current_state = next_state
            
            # Terminate the episode if a terminal state is reached (you should replace this with your termination condition)
        cumr+=sess_reward
    return cumr, training   


In [156]:
def evaluatePolicy(policy, Qt =[],numSamples=100, numTrans=2000):
    c = 0
    for i in range(numSamples):
        cum_r, train = run_episodes(Qt,policy,numTrans)
        c = c+cum_r
    
    return c/numSamples

In [157]:
def q_learning(Q_table, training, alpha, gamma, test=False):
    # Calculate the Q-value for the current state-action pair
    t=0
    for sample in training:
        current_state,slate_action,next_state,reward_value=sample
        action=slates.index(slate_action)
        state=user_states.index(current_state)
        current_q_value = Q_table[state, action]

    # Find the maximum Q-value for the next state and all possible actions
        next_state = user_states.index(next_state)
        max_next_q_value = np.max(Q_table[next_state, :])

    # Update the Q-value using the Q-learning formula
    
        new_q_value = current_q_value + alpha * (reward_value + gamma * max_next_q_value - current_q_value)

    # Update the Q-table with the new Q-value
        Q_table[state, action] = new_q_value
        if test and (t%500)==0:
            v=evaluatePolicy(qLearned,Q_table)
            action=np.argmax(Q_table,axis=1)
            print(action, end='')
            print(t, '\t',v)
        t=t+1

    return Q_table

def slateqLearning(Q_item,Q_slate,training, alpha, gamma,test=False):
    t=0
    for sample in training:
        current_state,selected_item,selected_slate,reward_value,next_state=sample
        state=user_states.index(current_state)
        action=slates.index(selected_slate)
        if selected_item!=0:
            selected_item_index=items.index(selected_item)
            current_q_value=Q_item[state,selected_item_index]
            next_state=user_states.index(next_state)
            max_next_q_value=np.max(Q_item[next_state,:])
            new_q_value=current_q_value+alpha*(reward_value+gamma*max_next_q_value-current_q_value)
            Q_item[state,selected_item_index]=new_q_value
        
        for i in range(len(selected_slate)):
            item_action=items.index(selected_slate[i])
            Q_slate[state,action]+=Q_item[state,item_action]*prob_i_u[state,item_action]
        if test and (t%500)==0:
            v=evaluatePolicy(qLearned,Q_slate)
            action=np.argmax(Q_slate,axis=1)
            print(action, end='')
            print(t, '\t',v)
        t=t+1

    return Q_slate

In [158]:
cum_r, training=run_episodes(Q_table,randSlate, 200, 10, transition_function, reward, 0.1)
cum_rslate, training_slate=run_episodes_slateq(Q_slate,randSlate, 200, 10, transition_function_slateq, reward_slateq, 0.1)

In [159]:
qtable = q_learning(Q_table, training, 0.01, 0.9,True)
qslate=slateqLearning(Q_item,Q_slate,training_slate, 0.01, 0.9,True)

[0 0 0]0 	 18661.84
[20 30 14]500 	 20000.0
[10 28  7]1000 	 20000.0
[10 30 30]1500 	 20000.0
[ 8 11 31]499 	 20000.0
[ 4 10 31]999 	 20000.0
[ 2 30 29]1499 	 20000.0
[ 2 30 27]1999 	 20000.0


In [160]:
first_max_slateq = np.argmax(Q_slate, axis=1)
second_max_slateq= np.argsort(Q_slate, axis=1)[:,-2]
# Print the result
for i in range(len(first_max_slateq)):
    print(first_max_slateq[i],second_max_slateq[i])
    print(f"SlateQ : For user state {user_states[i]}: optimal slate is {slates[first_max_slateq[i]]} and {slates[second_max_slateq[i]]} ")

first_max = np.argmax(Q_table, axis=1)
second_max= np.argsort(Q_table, axis=1)[:,-2]
# Print the result
for i in range(len(first_max)):
    print(first_max[i],second_max[i])
    print(f"For user state {user_states[i]}: optimal slate is {slates[first_max[i]]} and {slates[second_max[i]]} ")

2 17
SlateQ : For user state -1: optimal slate is (-1, -1, -1, 1, -1) and (1, -1, -1, -1, 1) 
30 29
SlateQ : For user state 0: optimal slate is (1, 1, 1, 1, -1) and (1, 1, 1, -1, 1) 
27 31
SlateQ : For user state 1: optimal slate is (1, 1, -1, 1, 1) and (1, 1, 1, 1, 1) 
10 8
For user state -1: optimal slate is (-1, 1, -1, 1, -1) and (-1, 1, -1, -1, -1) 
28 30
For user state 0: optimal slate is (1, 1, 1, -1, -1) and (1, 1, 1, 1, -1) 
11 30
For user state 1: optimal slate is (-1, 1, -1, 1, 1) and (1, 1, 1, 1, -1) 


In [161]:
Q_slate

array([[ 97.06265154, 134.20386477, 200.39180529, 114.51860615,
        131.84563733, 128.64697638,  94.84303486,  66.77602982,
        108.66970774, 137.21522675,  92.56431787,  55.19487258,
         98.91897868,  95.03465239,  40.79988025,  29.90246104,
         66.02659434, 148.42295847, 101.16903899,  63.77385998,
        116.21032747,  58.24310875,  50.41608215,  33.49603728,
         82.7175868 ,  77.4449918 ,  84.8736808 ,  25.44540501,
         67.0141909 ,  44.21339077,  32.43722773,   5.18173043],
       [  2.3466968 ,   7.25457025,   5.66020792,  15.36259102,
          7.44343251,   9.05083127,   4.64930762,  10.28629298,
          4.77347788,   5.87055032,   6.26978753,   9.62776109,
          5.60290676,  20.21735062,  16.52312703,  10.5930778 ,
          5.10199105,  14.65099847,  14.92846088,   8.85435146,
         11.2427293 ,  10.46643868,  17.77626836,  14.2315089 ,
         13.5320165 ,  10.3047418 ,  14.69926951,  12.89342238,
         12.63376872,  21.24401675,  22

In [162]:
Q_table

array([[ 0.22228268,  0.18961471,  0.27284294,  0.20869637,  0.24407476,
         0.24003799,  0.17441526, -0.17393003,  0.30921101,  0.20776096,
         0.41317404, -0.21392595,  0.19152398, -0.15884479, -0.17903279,
        -0.1924692 ,  0.22859408,  0.2788797 ,  0.30183444, -0.17964276,
         0.24846346, -0.15515513, -0.19471306, -0.18481839,  0.28586088,
        -0.16194038, -0.17370337, -0.15229613, -0.25242947, -0.1446992 ,
        -0.14858333, -0.17335872],
       [ 0.17061861,  0.1885507 ,  0.12252921,  0.16842303,  0.18863508,
         0.15632043,  0.16778235,  0.15225874,  0.16220515,  0.17948339,
         0.15266464,  0.18100349,  0.1548293 ,  0.14651199,  0.19157385,
         0.16981999,  0.15352388,  0.13623292,  0.1445608 ,  0.19001307,
         0.17480202,  0.18298563,  0.14267529,  0.17921997,  0.11411906,
         0.13902975,  0.19262131,  0.17319612,  0.2379022 ,  0.18067681,
         0.23318608,  0.13316978],
       [-0.15468406, -0.1578489 , -0.19741523, -0.2448

In [163]:
ranked_Q_table = np.apply_along_axis(lambda row: np.argsort(row)[::-1].argsort() + 1, axis=1, arr=Q_table)
ranked_Q_slate = np.apply_along_axis(lambda row: np.argsort(row)[::-1].argsort() + 1, axis=1, arr=Q_slate)

Ordered_difference = ranked_Q_table - ranked_Q_slate
Ordered_difference

array([[ -1,  11,   5,   4,   3,   3,   2,   5,  -7,  10, -14,   7,   3,
          8,  -1,  -1, -11,   3,  -7,   5,   0,  -3,   5,   0, -13,   4,
          8, -12,  13,  -9, -11,  -9],
       [-17, -17,   4,  11, -17,  -1, -13,   5, -11, -15,  -2, -11,  -7,
         22,  -1,   0,  -7,  19,  19, -17,  -2,  -9,  23,   1,  20,  10,
         -6,   1, -13,   8,   1,  22],
       [-13,   4,  -3,   2,  -6,   6,  10,  -6,  -9,   0,  14, -17,  -4,
         -3,  -9,   0,  -4,   3,   3, -10,   7,   3,   0,   5,   8,   7,
         -8,   7,  -4,  10,  -6,  13]])

In [164]:
qslate

array([[ 97.06265154, 134.20386477, 200.39180529, 114.51860615,
        131.84563733, 128.64697638,  94.84303486,  66.77602982,
        108.66970774, 137.21522675,  92.56431787,  55.19487258,
         98.91897868,  95.03465239,  40.79988025,  29.90246104,
         66.02659434, 148.42295847, 101.16903899,  63.77385998,
        116.21032747,  58.24310875,  50.41608215,  33.49603728,
         82.7175868 ,  77.4449918 ,  84.8736808 ,  25.44540501,
         67.0141909 ,  44.21339077,  32.43722773,   5.18173043],
       [  2.3466968 ,   7.25457025,   5.66020792,  15.36259102,
          7.44343251,   9.05083127,   4.64930762,  10.28629298,
          4.77347788,   5.87055032,   6.26978753,   9.62776109,
          5.60290676,  20.21735062,  16.52312703,  10.5930778 ,
          5.10199105,  14.65099847,  14.92846088,   8.85435146,
         11.2427293 ,  10.46643868,  17.77626836,  14.2315089 ,
         13.5320165 ,  10.3047418 ,  14.69926951,  12.89342238,
         12.63376872,  21.24401675,  22